In [1]:
import os, json

with open('conf.json', 'r') as f:
        json_data = json.load(f)
    
os.environ['OPENAI_API_KEY'] = json_data['API_KEY']

In [2]:
from core.get_common_issues import get_news_dict_list

etf_tkr = "AIQ"

news_data = str(get_news_dict_list(etf_tkr=etf_tkr))

In [7]:
from data.fetch_data import fetch_data_from_db

import pandas as pd

sql = """
    select * from os_index_info
"""

etf_df = pd.DataFrame(fetch_data_from_db(query=sql))

etf_df.head()

,etf_tkr,index_name,upper_bound,extra_info,lower_bound,rebalance_period
0,AGNG,Indxx Aging Population Thematic Index,0.03,"3% 이상의 가중치를 가진 보안은 3%로 할당되고, 0.3% 이하의 가중치를 가진 ...",0.003,인덱스는 매년 한 번의 재구성과 재조정 일정을 따릅니다. 새로운 포트폴리오는 매년 ...
1,AIQ,Indxx Artificial Intelligence & Big Data Index,0.03,초과 가중치는 남은 제한이 없는 보안 중에서 비례하여 분배. 노출 점수가 20% 이...,0.001,\n\n인덱스는 매년 한 번의 재구성과 재조정 일정을 따릅니다. 새로운 포트폴리오는...
2,AQWA,Solactive Global Clean Water Industry Index,0.07,"7% 이상의 가중치를 가진 지수 구성요소의 총합 가중치는 40%로 제한, 나머지 종...",0.003,.
3,BFIT,Indxx Global Health & Wellness Thematic Index,0.03,"3% 이상의 가중치를 가진 보안은 3%로 할당되고, 0.3% 이하의 가중치를 가진 ...",0.003,인덱스는 매년 두 번의 재구성과 재조정 일정을 따릅니다. 인덱스는 매년 4월과 10...
4,BKCH,Solactive Blockchain Index,0.12,"4.5% 이상의 가중치를 가진 지수 구성요소의 총합 가중치는 45%로 제한, 나머지...",0.001,.


In [31]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI

from tqdm.auto import tqdm

def get_llm_answer(etf_tkr, text):
    template = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=(
                    f"""
                    Summarize the common issues based on the news headlines of stocks which are best 5 influential components of {etf_tkr} in Korean.
                    
                    - Tone : Humble
                    - Style : MECE, accurate
                    - Reader level : college student
                    - Length : within 500 characters
                    - Perspective : Mckinsey&Company senior consultant
                    - Format : markdown
                    
                    You MUST answer in Korean. 
                    """
                )
            ),
            HumanMessagePromptTemplate.from_template("{text}"),
        ]
    )

    llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
    
    ret = llm(template.format_messages(text=text))

    return ret

for i, row in tqdm(etf_df.iterrows()):
    
    etf_tkr = row.etf_tkr
    
    etf_name = row.index_name
    
    news_data_per_etf = str(get_news_dict_list(etf_tkr=etf_tkr))

    llm_answer = get_llm_answer(etf_tkr=etf_name, text=news_data_per_etf)

    print(llm_answer.content)
    
    print()

    with open(f'./etf_llm_summary/{etf_tkr}.json', 'w') as outfile:
        json.dump({"llm_answer": llm_answer.content}, outfile, indent='\t', ensure_ascii=False)

0it [00:00, ?it/s]

Indxx Aging Population Thematic Index의 가장 영향력 있는 구성 요소인 주식들에 대한 뉴스 헤드라인을 기반으로 일반적인 문제를 요약하면 다음과 같습니다.

1. 약제 회사들과의 협상에 대한 미국 메디케어 가격 협상 목록의 영향: 약제 회사들은 메디케어 가격 협상 목록에 영향을 받지 않았으며, 이에 관한 분석 보고서들이 발표되었습니다.

2. 고가 약물에 대한 미국 가격 협상: 고가 약물들이 미국에서 처음으로 가격 협상의 대상이 되었습니다.

3. 암 치료 시장의 잠재력: 암 치료 시장의 큰 잠재력으로 인해 암 치료 주식들에 대한 관심이 집중되고 있습니다.

4. 메디케어 가격 협상의 대형 약제 회사 주식에 미치는 영향: 메디케어 가격 협상이 대형 약제 회사 주식들에 어떤 영향을 미칠지에 대한 논란이 있습니다.

5. 주식 시장의 변동: 특정 회사들의 재무 성과나 기업 이벤트에 따라 주식 시장이 상승하거나 하락할 수 있습니다.

이러한 주요 문제들은 Indxx Aging Population Thematic Index의 구성 요소인 주식들에 영향을 미칠 수 있으며, 투자자들은 이러한 동향을 주시해야 합니다.

Indxx 인공지능 및 빅데이터 지수의 가장 영향력 있는 구성 요소인 주식들을 기반으로 한 뉴스 헤드라인을 통해 일반적인 문제를 요약하면 다음과 같습니다.

1. AAPL (애플)
- 지속 가능성 스타트업 2개를 선정한 애플의 Impact Accelerator
- 3D Systems 주식의 급등
- 애플 주식이 연간 최악의 달을 향하고 있다는 소식

2. GOOGL (구글)
- 개인정보 보호 활동가 Schrems가 Google의 Fitbit에 대해 불만 제기
- 인도와 일본에서 구글의 생성적 AI 검색 엔진 도입
- AI 서버 판매가 2024년까지 2배로 증가할 것이라고 예상

3. AMZN (아마존)
- 아마존 로지스틱스를 판매자에게 개방하는 아마존과 Shopify의 협약
- Shopify 상점에서 'Buy with Prime'